In [ ]:
cd data

In [ ]:

import cv2
import math
import numpy as np
from sympy.geometry import *
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from shapely.geometry import Polygon
from tqdm import tqdm
import pandas as pd
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import pyocr
import re
import array
import csv
from pylsd.lsd import lsd
import sys

In [ ]:
thresh = 50
N = 11
wndname = "Square Detection Demo"

# ベクトル間の角度の余弦(コサイン)を見つけます
# pt0-pt1およびpt0-pt2のなす角のコサインを取得
#
def angle(pt1, pt2, pt0) -> float:
    dx1 = float(pt1[0,0] - pt0[0,0])
    dy1 = float(pt1[0,1] - pt0[0,1])
    dx2 = float(pt2[0,0] - pt0[0,0])
    dy2 = float(pt2[0,1] - pt0[0,1])
    v = math.sqrt((dx1*dx1 + dy1*dy1)*(dx2*dx2 + dy2*dy2) )
    return (dx1*dx2 + dy1*dy2)/ v


# 画像上で検出された一連の正方形を返します。
#
def findSquares(image, squares, areaThreshold=10000000):
    squares.clear()
    gray0 = np.zeros(image.shape[:2], dtype=np.uint8)

    # down-scale and upscale the image to filter out the noise
    rows, cols, _channels = map(int, image.shape)
    pyr = cv2.pyrDown(image, dstsize=(cols//2, rows//2))
    timg = cv2.pyrUp(pyr, dstsize=(cols, rows))

    # 画像のBGRの色平面で正方形を見つける
    for c in range(0, 3):
        cv2.mixChannels([timg], [gray0], (c, 0))

        # いくつかのしきい値レベルを試す
        for l in range(0, N):
#            print('l=%d' % (l))

            # l:ゼロしきい値レベルの代わりにCannyを使用します。
            # Cannyはグラデーションシェーディングで正方形を
            # キャッチするのに役立ちます
            if l == 0:
                # Cannyを適用
                # スライダーから上限しきい値を取得し、下限を0に設定します
                # （これによりエッジが強制的にマージ）
                #
                gray = cv2.Canny(gray0,thresh, 5)

                #Canny出力を拡張して、エッジセグメント間の潜在的な穴を削除します
                gray = cv2.dilate(gray, None)
            else:
                # apply threshold if l!=0:
                gray[gray0 >= (l+1)*255/N] = 0
                gray[gray0 < (l+1)*255/N] = 255

            # 輪郭をリストで取得
            contours, _ = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for i, cnt in enumerate(contours):
                # 輪郭の周囲を取得
                arclen = cv2.arcLength(cnt, True)

                # 輪郭の近似
                approx = cv2.approxPolyDP(cnt, arclen*0.02, True)

                # 面積
                area = abs(cv2.contourArea(approx))

                #長方形の輪郭は、近似後に4つの角をもつ、
                #比較的広い領域
                #（ノイズの多い輪郭をフィルターで除去するため）
                #凸性(isContourConvex)になります。
                if approx.shape[0] == 4 and area > areaThreshold and cv2.isContourConvex(approx) :
                    maxCosine = 0

                    for j in range(2, 5):
                        # ジョイントエッジ間の角度の最大コサインを見つけます
                        cosine = abs(angle(approx[j%4], approx[j-2], approx[j-1]))
                        maxCosine = max(maxCosine, cosine)

                    # すべての角度の余弦定理が小さい場合（すべての角度が約90度）、
                    # 結果のシーケンスにquandrange頂点を書き込みます
                    if maxCosine < 0.2 :
                        squares.append(approx)



def find_map_extent(squares):
    
    list_of_areas=[]
    list_of_polygons=[]

    for i in tqdm(range(len(squares))):
        coords=[(squares[i].reshape(-1)[2*j],squares[i].reshape(-1)[2*j+1]) for j in range(int(len(squares[1].reshape(-1))/2))]
        coords.append(coords[0])
        coords=tuple(coords)
        list_of_areas.append(Polygon(coords).area)
        list_of_polygons.append(Polygon(coords))

    df=pd.DataFrame({'areas':list_of_areas,'polygons':list_of_polygons})
    #thresh=1e7

    #large_blocks=df[df['areas']>thresh]
    #map_extent=large_blocks.sort_values('areas',ascending=False).iloc[-1] 
    #面積の最も小さいものを選び出す

    map_extent=df.sort_values('areas',ascending=False).iloc[-1] 
    #x,y =map_extent['polygons'].exterior.xy
    return map_extent


def find_map_corners(map_extent,window_size,fp):
    
    x,y=map_extent['polygons'].exterior.xy
    image = cv2.imread(fp,cv2.IMREAD_COLOR)

    corner_images=[]
    corner_images_raw=[]
    
    for i in range(4):
        clipped=image[int(y[i]-window_size/2):int(y[i]+window_size/2),int(x[i]-window_size/2):int(x[i]+window_size/2),:]
        corner_images_raw.append(clipped)
        im_gray = cv2.cvtColor(clipped,cv2.COLOR_BGR2GRAY)
        corner_images.append(im_gray)
        
        return corner_images,corner_images_raw
    

def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(15, 15))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        try:
            image=np.moveaxis(np.array([image[:,:,2:3],image[:,:,1:2],image[:,:,0:1]]),[0,1,2],[2,0,1])[:,:,:,0]/3000
            plt.imshow(image)
        except:
            plt.imshow(image,cmap='gray')
        plt.show()
        
        
def tate(x,y):
    
    #find_map_extentで求められた四隅のデータを、次のdef文に合わせるよう処理します。
    point = []
    for i in range(0,4):
        point.append(x[i])
    point = np.array([point],dtype='int16')
    pointx = np.swapaxes(point,0,1)

    point = []
    for i in range(0,4):
        point.append(y[i])
    point = np.array([point],dtype='int16')
    pointy = np.swapaxes(point,0,1)
    
    point = np.append(pointx,pointy,axis=1)
    return point


def sort4(test):

    #四隅のデータを、左上、左下、右上、右下、の順に並べます
    x = []

    for i in range(len(test)):
        x.append(test[i,0]**2+test[i,1]**2)
    
    x  = np.array([x])
    x = np.swapaxes(x,0,1)
        
    test2 = np.append(test, x, axis=1)
        
    col_num=2
    test3 = test2[np.argsort(test2[:,col_num])]
    test4= test3[:,0:2]
    return test4


#ここまでで四隅1が出る（嶌田さんプログラム）



        


In [ ]:
#ここから四隅推定となる（Jupyter Notebook枠の、4～8まで）
#外枠用四隅の検出のためのテンプレート画像の設定
tempLU = cv2.imread('tempLU.jpg')
tempRU = cv2.imread('tempRU.jpg')
tempLD = cv2.imread('tempLD.jpg')
tempRD = cv2.imread('tempRD.jpg')
#外枠の4隅用

tempLLU = cv2.imread('tempLLU.jpg')
tempLRU = cv2.imread('tempLRU.jpg')
tempLLD = cv2.imread('tempLLD.jpg')
tempLRD = cv2.imread('tempLRD.jpg')
#緯度経度枠

h,w,c = tempLU.shape

In [ ]:
#緯度経度点の第一次近似値を求める

def pointLU(img_gray,tempLU,h,w):
    ha,wa = img_gray.shape
    imgcLU = img_gray[0:int(ha/2),0:int(wa/2)]
    tempLUG = cv2.cvtColor(tempLU, cv2.COLOR_BGR2GRAY)
    resLU = cv2.matchTemplate(imgcLU, tempLUG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLU)
    locLU = np.where(resLU >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLU)
    top_left = max_loc
    x = top_left[0]+h/2+0
    y = top_left[1]+w/2+0
    X = x + 166
    Y = y + 166
#    print(threshold)
    return(X,Y)



def pointLD(img_gray,tempLD,h,w):
    ha,wa = img_gray.shape
    imgcLD = img_gray[int(ha/2):ha,0:int(wa/2)]
    tempLDG = cv2.cvtColor(tempLD, cv2.COLOR_BGR2GRAY)
    resLD = cv2.matchTemplate(imgcLD, tempLDG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLD)
    locLD = np.where(resLD >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLD)
    top_left = max_loc
    x = top_left[0]+h/2+0
    y = top_left[1]+w/2+ha/2
    X = x + 166
    Y = y - 166
#    print(threshold)
    return(X,Y)



def pointRU(img_gray,tempRU,h,w):
    ha,wa = img_gray.shape
    imgcRU = img_gray[0:int(ha/2),int(wa/2):wa]
    tempRUG = cv2.cvtColor(tempRU, cv2.COLOR_BGR2GRAY)
    resRU = cv2.matchTemplate(imgcRU, tempRUG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resRU)
    locRU = np.where(resRU >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resRU)
    top_left = max_loc
    x = top_left[0]+h/2+int(wa/2)
    y = top_left[1]+w/2+0
    X = x - 166
    Y = y + 166
#    print(threshold)
    return(X,Y)



def pointRD(img_gray,tempRD,h,w):
    ha,wa = img_gray.shape
    imgcRD = img_gray[int(ha/2):ha,int(wa/2):wa]
    tempRDG = cv2.cvtColor(tempRD, cv2.COLOR_BGR2GRAY)
    resRD = cv2.matchTemplate(imgcRD, tempRDG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resRD)
    locRD = np.where(resRD >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resRD)
    top_left = max_loc
    x = top_left[0]+h/2+int(wa/2)
    y = top_left[1]+w/2+int(ha/2)
    X = x - 166
    Y = y - 166
#    print(threshold)
    return(X,Y)


In [ ]:
#大本の画像を処理して、垂直線、水平線だけを残す
def check1(img3,lines4,lo1,hi1): 
    
    for lines4_cont in lines4:
        x1,y1,x2,y2 = map(int,lines4_cont)
        t = math.atan2(y2-y1,x2-x1)
        if (lo1< abs(t)):
            if (hi1> abs(t)):
                img3 = cv2.line(img3,(x1,y1),(x2,y2),(255,255,255),1)
            else:
                pass
        else:
            pass
    return img3


def check2(img3,lines4,da):

    for lines4_cont in lines4:
        x1,y1,x2,y2 = map(int,lines4_cont)
        t = math.atan2(y2-y1,x2-x1)
        if (da > abs(t)):
            img3 = cv2.line(img3,(x1,y1),(x2,y2),(255,255,255),1)
        else:
            pass
    return img3



In [ ]:
#緯度経度点周辺でのテンプレートマッチング（緯度経度枠）と、それによる緯度経度点の推定
def pointLLU(img3cc,tempLLU):
    tempLLUG = cv2.cvtColor(tempLLU, cv2.COLOR_BGR2GRAY)
    resLLU = cv2.matchTemplate(img3cc, tempLLUG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLLU)
    locLU = np.where(resLLU >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLLU)
    top_left = max_loc
    x = top_left[0]
    y = top_left[1]

    return(x,y,threshold)

def pointLLD(img3cc,tempLLD):
    tempLLDG = cv2.cvtColor(tempLLU, cv2.COLOR_BGR2GRAY)
    resLLD = cv2.matchTemplate(img3cc, tempLLDG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLLD)
    locLD = np.where(resLLD >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLLD)
    top_left = max_loc
    x = top_left[0]
    y = top_left[1]

    return(x,y,threshold)


def pointLRU(img3cc,tempLRU):
    tempLRUG = cv2.cvtColor(tempLRU, cv2.COLOR_BGR2GRAY)
    resLRU = cv2.matchTemplate(img3cc, tempLRUG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLRU)
    locRU = np.where(resLRU >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLRU)
    top_left = max_loc
    x = top_left[0]
    y = top_left[1]

    return(x,y,threshold)

def pointLRD(img3cc,tempLRU):
    tempLRDG = cv2.cvtColor(tempLRD, cv2.COLOR_BGR2GRAY)
    resLRD = cv2.matchTemplate(img3cc, tempLRDG, cv2.TM_CCOEFF_NORMED)
    threshold = np.amax(resLRD)
    locRD = np.where(resLRD >= threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resLRD)
    top_left = max_loc
    x = top_left[0]
    y = top_left[1]

    return(x,y,threshold)



In [ ]:
#テンプレートマッチングの2方式の結果、どちらの緯度経度点を使用するかを決める
#pointD = []

def yosumi(img_gray,h,w,tempLU,tempLD,tempRU,tempRD,tempLLU,tempLLD,tempLRU,tempLRD):
    pointD=[]
    
    for i,xx in enumerate(('LU','LD','RU','RD')):
        point_func1 = getattr(sys.modules[__name__], 'point{}'.format(xx))
        point_func2 = getattr(sys.modules[__name__], 'pointL{}'.format(xx))

        X,Y = point_func1(img_gray,locals()['temp{}'.format(xx)],h,w)
        Xi = int(X)
        Yi = int(Y)

        #print(Xi,Yi)
        img2 = img_gray[Yi-150:Yi+150, Xi-150: Xi+150]
        img2r = cv2.rotate(img2, cv2.ROTATE_90_CLOCKWISE)
        size = img2.shape
        img3 = np.zeros(size,dtype=np.uint8)

        da = math.pi*10/360
        hi1 = math.pi/2.0+da
        lo1 = math.pi/2.0-da

        linesL = lsd(img2)
        lines4 = linesL[:,0:4]

        linesLr = lsd(img2r)
        lines4r = linesLr[:,0:4]

        img3s = check1(img3,lines4,lo1,hi1)
        img4s = check2(img3s,lines4,da)

        x,y,thr = point_func2(img4s,locals()['tempL{}'.format(xx)])
#        print(x,y,thr)

        img4c = cv2.rotate(img4s, cv2.ROTATE_90_CLOCKWISE)

        img3r = check1(img4c,lines4r,lo1,hi1)
        img4r = check2(img3r,lines4,da)
        img4rc = cv2.rotate(img4r, cv2.ROTATE_90_COUNTERCLOCKWISE)
        xr,yr,thrr = point_func2(img4rc,locals()['tempL{}'.format(xx)])
#        print(xr,yr,thrr)

        if (thr >= 0.6):
            if (thrr >= 0.6):
                xL = Xi-150+xr+25
                yL = Yi-150+yr+25
            elif (thrr < 0.6):
                xL = Xi-150+x+25
                yL = Yi-150+y+25  
        elif (thrr >= 0.6):
                xL = Xi-150+xr+25
                yL = Yi-150+yr+25  
        else:
            xL = Xi
            yL = Yi

#        print(xL,yL)   
        pointD.append([xL,yL])
        pointDn = np.array(pointD) 
        
    return pointDn

#ここまで(4～8)が、テンプレートマッチング邦で緯度経度点の4点の座標値を検出する為のプログラム

In [ ]:
def compare2p(point1,point2):
    distance = np.linalg.norm(point1 - point2, axis=1)
    point3 = [[] for _ in range(4)]
#    print(distance)
    
    for j in range(4):
        if distance[j] >= 100:
            point3[j] = point2[j][:]
#            print("2")
        else:
            point3[j] = point1[j][:]
#            print("1")

        
    point3e = np.array(point3) 
#    print(point3e)

    return point3e

#嶌田さんプログラムとテンプレートマッチング法の、どちらの緯度経度を使うか決定する.
#具体的にどちらの方法を使っているかを確認するには、途中のprint("1"),print("2")を使う

In [ ]:
def check_empty_list(lst):
    if any(not sublst for sublst in lst):
        return [[]] * len(lst)
    else:
        return lst
#緯度経度の判読の空白が一か所でもあった場合、
#緯度経度点が正確に判読されていないと判断し、その地図全体の緯度経度を空白とする為の準備

In [ ]:
#OCRによる判読プログラム

def ocr(im, point,fp):
      #環境変数「PATH」にTesseract-OCRのパスを設定。
      #Windowsの環境変数に設定している場合は不要。
    path='C:\\Program Files\\Tesseract-OCR\\'
    os.environ['PATH'] = os.environ['PATH'] + path

      #pyocrにTesseractを指定する。
    pyocr.tesseract.TESSERACT_CMD = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    tools = pyocr.get_available_tools()
    tool = tools[0]
    

    Iul = im.crop((point[0,0]-160,point[0,1]-40,point[0,0],point[0,1]+40))
    Kul = im.crop((point[0,0]-50,point[0,1]-170,point[0,0]+80,point[0,1]))
    Kulc = Kul.rotate(270,expand=True)
    Idl = im.crop((point[1,0]-160,point[1,1]-40,point[1,0],point[1,1]+40))
    Kdl = im.crop((point[1,0]-50,point[1,1],point[1,0]+80,point[1,1]+170))
    Kdlc = Kdl.rotate(270,expand=True)
    Iur = im.crop((point[2,0],point[2,1]-40,point[2,0]+160,point[2,1]+40))
    Kur = im.crop((point[2,0]-50,point[2,1]-170,point[2,0]+80,point[2,1]))
    Kurc = Kur.rotate(270,expand=True)
    Idr = im.crop((point[3,0],point[3,1]-40,point[3,0]+160,point[3,1]+40))
    Kdr = im.crop((point[3,0]-50,point[3,1],point[3,0]+80,point[3,1]+170))
    Kdrc = Kdr.rotate(270,expand=True)
    
    row = 2 # グラフの行数
    col = 4 # グラフの列数

    fig,ax=plt.subplots(row, col, figsize=(16,3))
    fig.suptitle(fp)
    ax[0,0].imshow(Iul)
    ax[0,1].imshow(Idl)
    ax[0,2].imshow(Iur)
    ax[0,3].imshow(Idr)
    ax[1,0].imshow(Kulc)
    ax[1,1].imshow(Kdlc)
    ax[1,2].imshow(Kurc)
    ax[1,3].imshow(Kdrc)
    fp_name = os.path.splitext(os.path.basename(fp))[0]
    plt.savefig(f'E:/Japan/Moji/fig{fp_name}.jpg')  
    plt.close()
#文字判読に使用した画像をまとめて一つの画像データとする。この為のフォルダを先に作成しておく。
    
    builder = pyocr.builders.TextBuilder(tesseract_layout=11)
    tIul = tool.image_to_string(Iul, lang="eng", builder=builder)
    nIul=re.compile(r"\d+").findall(tIul)
    tKul = tool.image_to_string(Kulc, lang="eng", builder=builder)
    nKul=re.compile(r"\d+").findall(tKul)

    tIdl = tool.image_to_string(Idl, lang="eng", builder=builder)
    nIdl=re.compile(r"\d+").findall(tIdl)
    tKdl = tool.image_to_string(Kdlc, lang="eng", builder=builder)
    nKdl=re.compile(r"\d+").findall(tKdl)

    tIur = tool.image_to_string(Iur, lang="eng", builder=builder)
    nIur=re.compile(r"\d+").findall(tIur)
    tKur = tool.image_to_string(Kurc, lang="eng", builder=builder)
    nKur=re.compile(r"\d+").findall(tKur)

    tIdr = tool.image_to_string(Idr, lang="eng", builder=builder)
    nIdr=re.compile(r"\d+").findall(tIdr)
    tKdr = tool.image_to_string(Kdrc, lang="eng", builder=builder)
    nKdr=re.compile(r"\d+").findall(tKdr)
        
    nIul_s = ''.join(nIul)
    nKul_s = ''.join(nKul) 
    nIdl_s = ''.join(nIdl)
    nKdl_s = ''.join(nKdl) 
    nIur_s = ''.join(nIur)
    nKur_s = ''.join(nKur) 
    nIdr_s = ''.join(nIdr)
    nKdr_s = ''.join(nKdr)

    latmat = [nIul_s,nIdl_s,nIur_s,nIdr_s]
    lonmat = [nKul_s,nKdl_s,nKur_s,nKdr_s]
    
    latmat = check_empty_list(latmat)
    lonmat = check_empty_list(lonmat)
    
    return latmat,lonmat



In [ ]:
#判読した緯度経度値から、明らかな異常値を捨てる

def Latt(lat):
    Lat = str(lat)
    if len(Lat) < 3:
        Lat = 0
    elif len(Lat)<4:
        if int(Lat[2:3])!= 0:
            Lat = 0
        elif int(Lat[:2])<=24:
            Lat = 0
        elif int(Lat[:2])>=46:
            Lat = 0
        elif int(Lat[2:3])>=6:
            Lat = 0
        else:
            Lat = int(Lat+'0')
    elif int(Lat[3:]) != 0:
        Lat = 0
    elif int(Lat[:2])<=24:
        Lat = 0
    elif int(Lat[:2])>=46:
        Lat = 0
    elif int(Lat[2:3])>=6:
        Lat = 0
    else:
        Lat = int(Lat)
    return Lat
#緯度24度以下、もしくは46度以上ならば異常値と見なす。
#この値は適宜変えること

def Long(lon):
    Lon = str(lon)
    num = len(Lon)
    if num < 4:
        Lon = 0
    elif num < 5:
        if int(Lon[:3])<=122:
            Lon = 0
        elif int (Lon[:3])>=148:
            Lon = 0
        elif int(Lon[3:4])!= 0:
            Lon = 0
        else:
            Lon =int(Lon+'0')
    elif num >= 5:
        if int(Lon[:3])<=122:
            Lon = 0
        elif int (Lon[:3])>=148:
            Lon = 0    
        elif (Lon[3:5]) not in ['01','15','30','45']:
            Lon = 0
        elif Lon[3:5] == '01':
            Lon = int(Lon[0:4]+'0')
        else:
            Lon = int(Lon[0:5])
    return Lon
#経度122度以下、もしくは148度以上ならば異常値と見なす。
#この値は適宜変えること


In [ ]:
#残った緯度値から、四隅の緯度の値を決める

def latmake1(lu):
    if lu >= 1000:
        if int(str(lu)[2:]) != 0:
            lu1 = lu
            ld1 = int(str(int(str(lu)[:2]))+str(int(str(lu)[2:])-10))
            ru1 = lu
            rd1 = int(str(int(str(lu)[:2]))+str(int(str(lu)[2:])-10))
        else:
            lu1 = lu
            ld1 = int(str(int(str(lu)[:2])-1)+'50')
            ru1 = lu   
            rd1 = int(str(int(str(lu)[:2])-1)+'50')
    else:
        lu1 = 0
        ru1 = 0
        ld1 = 0
        rd1 = 0

    mat = np.array((lu1,ld1,ru1,rd1))
    return mat

def latmake2(ld):
    if ld >= 1000:
        if int(str(ld)[2:]) != 50:
            lu1 = int(str(int(str(ld)[:2]))+str(int(str(ld)[2:])+10))
            ru1 = int(str(int(str(ld)[:2]))+str(int(str(ld)[2:])+10))
            ld1 = ld
            rd1 = ld
        else:
            lu1 = int(str(int(str(ld)[:2])+1)+'00')
            ru1 = int(str(int(str(ld)[:2])+1)+'00')
            ld1 = ld
            rd1 = ld
    else:
        lu1 = 0
        ru1 = 0
        ld1 = 0
        rd1 = 0

    mat = np.array((lu1,ld1,ru1,rd1))
    return mat



In [ ]:
#残った経度値から、四隅の経度値を決める

def lonmake1(lu):
    if lu >= 1000:
        if int(str(lu)[3:]) != 45:
            lu1 = lu
            ru1 = int(str(int(str(lu)[:3]))+str(int(str(lu)[3:5])+15))
            ld1 = lu
            rd1 = int(str(int(str(lu)[:3]))+str(int(str(lu)[3:5])+15))
        else:
            lu1 = lu
            ru1 = int(str(int(str(lu)[:3])+1)+'00')  
            ld1 = lu
            rd1 = int(str(int(str(lu)[:3])+1)+'00')
    else:
        lu1 = 0
        ru1 = 0
        ld1 = 0
        rd1 = 0

    mat = np.array((lu1,ld1,ru1,rd1))
    return mat

def lonmake2(ru):
    if ru >= 1000:
        if int(str(ru)[3:]) == 0:
            lu1 = int(str(int(str(ru)[:3])-1)+'45')  
            ru1 = ru 
            ld1 = int(str(int(str(ru)[:3])-1)+'45')  
            rd1 = ru
        elif int(str(ru)[3:]) ==15:
            lu1 = int(str(int(str(ru)[:3]))+'00')
            ru1 = ru
            ld1 = int(str(int(str(ru)[:3]))+'00')
            rd1 = ru
        else:
            lu1 = int(str(int(str(ru)[:3]))+str(int(str(ru)[3:5])-15))
            ru1 = ru
            ld1 = int(str(int(str(ru)[:3]))+str(int(str(ru)[3:5])-15))
            rd1 = ru
    else:
        lu1 = 0
        ru1 = 0
        ld1 = 0
        rd1 = 0

    mat = np.array((lu1,ld1,ru1,rd1))
    return mat



In [ ]:
#作成した緯度値、経度値をまとめる

def soroe(latmat,lonmat):
    Mlat = []
    for i in range(4):
        a = Latt(latmat[i])
        Mlat.append(a)
    
    Mlon =[]
    for i in range(4):
        b = Long(lonmat[i])
        Mlon.append(b)
    return Mlat,Mlon



In [ ]:
#まとめた緯度値を各位置（左上、左下、右上、右下））で比較し、全て同じでない場合は『解なし（0）』とした。
#また、全て同じ値の場合は、その値を各位置の緯度とした。

def makelat(Mlat):
    
    mlat1 = latmake1(Mlat[0])
    mlat2 = latmake2(Mlat[1])
    mlat3 = latmake1(Mlat[2])
    mlat4 = latmake2(Mlat[3])
    matrix = [mlat1,mlat2,mlat3,mlat4]
#    print(matrix)
    non_zero_rows = [i for i, row in enumerate(matrix) if not np.all(row == 0)]
#    print(non_zero_rows)

    if non_zero_rows !=[]:
        matrix2 = np.array([matrix[i] for i in non_zero_rows])
#    print(matrix2)
        if all(row[0] == matrix2[0][0] for row in matrix2) and all(row[2] == matrix2[0][2] for row in matrix2):
            lat4 = matrix2[0]
        else:
            lat4 = '0000'
    else:
        lat4 = '0000'
    
    lat4_b = [str(num)[:2]+'°'+str(num)[2:]+'′' for num in lat4]
    
    return lat4,lat4_b



In [ ]:
#まとめた経度値を各位置（左上、左下、右上、右下））で比較し、全て同じでない場合は『解なし（0）』とした。
#また、全て同じ値の場合は、その値を各位置の経度とした。

def makelon(Mlon):

    mlon1 = lonmake1(Mlon[0])
    mlon2 = lonmake1(Mlon[1])
    mlon3 = lonmake2(Mlon[2])
    mlon4 = lonmake2(Mlon[3])
    matrixL = [mlon1,mlon2,mlon3,mlon4]
#    print(matrixL)
    non_zero_rowsL = [i for i, row in enumerate(matrixL) if not np.all(row == 0)]
#    print(non_zero_rowsL)
    if non_zero_rowsL !=[]:
        matrix2L = np.array([matrixL[i] for i in non_zero_rowsL])
        if all(row[0] == matrix2L[0][0] for row in matrix2L) and all(row[2] == matrix2L[0][2] for row in matrix2L):
            lon4 = matrix2L[0]
        else:
            lon4 = '0000'
    else:
        lon4 = '0000'

    lon4_b = [str(num)[:3]+'°'+str(num)[3:]+'′'+'10.4″' for num in lon4]
    return lon4,lon4_b

#経度の10.4″はここで一律に付与している。10.4″が必要ない場合はlon4_bで変えること

In [ ]:
#画像の四隅の点の座標を与えて、地図範囲を切り取る  

def maskim(src_img,point,fp):
    mask = np.zeros_like(src_img) # (y, x, c)
    test_int = point.astype(np.int32)
    mask_poly = np.array([test_int[0],test_int[1],test_int[3],test_int[2]])
    mask = cv2.fillConvexPoly(mask, np.array(mask_poly, 'int32'), color=(255, 255, 255))
    masked_src_img = np.where(mask==255, src_img, mask)
    
    minx = np.amin(mask_poly[:,0])
    maxx = np.amax(mask_poly[:,0])
    miny = np.amin(mask_poly[:,1])
    maxy = np.amax(mask_poly[:,1])

    a = minx.tolist()
    b = maxx.tolist()
    c = miny.tolist()
    d = maxy.tolist()
    
    im_crop =masked_src_img[c:d,a:b]


    return im_crop
    



In [ ]:
import glob

files = glob.glob("E:/Japan/Hokkaido/original/test/*.jpg")
#処理する外邦図のフォルダを指定する

app = np.array([[2,2],
                [2,-2],
                [-2,2],
                [-2,-2]])

app_c = np.array([[-100,-100],
                  [-100,100],
                  [100,-100],
                  [100,100]])

app_C = np.array([[100,100],
                  [100,100],
                  [100,100],
                  [100,100]]) 


for fp in files:
        print(fp)
        image = cv2.imread(fp, cv2.IMREAD_COLOR)
        squares =[]
        findSquares(image, squares, areaThreshold=10000000)
        map_extent=find_map_extent(squares)
        x,y =map_extent['polygons'].exterior.xy
        point = sort4(tate(x,y))
        point2 = point-app 
#        嶌田さんプログラムに四隅の座標
        
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        pointD =yosumi(img_gray,h,w,tempLU,tempLD,tempRU,tempRD,tempLLU,tempLLD,tempLRU,tempLRD)
#        テンプレートンマッチングによる四隅の座標
        
        point3 = compare2p(point2,pointD) 
        point_cut = point3+app_c
#        print(point3)
#       この二つを比較した結果、どちらを緯度経度点の座標として使うかを決める
        
        im = Image.open(fp)   
        latmat,lonmat =  ocr(im, point3,fp)
        Mlat,Mlon = soroe(latmat,lonmat)
        lat4,lat4_b = makelat(Mlat)
        lon4,lon4_b = makelon(Mlon)
#        print(lat4_b)
#        print(lon4_b)
        pointa = np.array(point3)
        pointC = pointa-np.min(pointa,axis = 0)
        pointC_C = pointC+app_C
#        print(pointC)

        data = np.column_stack((pointC_C, lat4_b, lon4_b))
    
        bname = os.path.splitext(os.path.basename(fp))[0]
        
        if lat4 == '0000':         
            with open(f"E:/Japan/result2/cood{bname}.csv", 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(data)        
        elif lon4 =='0000' :
            with open(f"E:/Japan/result2/cood{bname}.csv", 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(data)        
        else:
            with open(f"E:/Japan/result1/cood{bname}.csv", 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(data)                
        
#四隅の座標と緯度経度値を組み合わせたcoodファイル(csvファイル)を作る
#四隅の検出と文字の判読が出来ていれば、folder(result1）に、出来ていない場合はfolder(result2)に入れる.
        
        
        im_crop = maskim(image,point_cut,fp)
        height, width, channels = im_crop.shape[:3]
#        bname = os.path.splitext(os.path.basename(fp))[0]
        print(bname)   
#        with open('G:/gaihozu/gaihozu_jpg/JPG/test61_copy/result/yosumi.csv',mode='a', encoding='utf-8') as fout:
#            writer = csv.writer(fout)
#            writer.writerow([bname])
#            writer.writerow([height,width])
#            writer.writerow([point3])
        cv2.imwrite(f"E:/Japan/result1/res_{bname}.jpg", im_crop)
        cv2.waitKey(0)
        cv2.destroyAllWindows()